<h1 style="font-size:42px; text-align:center; margin-bottom:30px;"><span style="color:SteelBlue">Cubank:</span> Machine Learning with Python and TM1</h1><hr>

## Import Dependencies

Before we get started, plotting libraries need to be imported. All the other dependencies we can import on the fly when needed.

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py

py.init_notebook_mode()

%matplotlib inline

# Part 1: Bring TM1 Data To Python

-----

## 1.1 Query TM1 cube data

We use **TM1py** to query data from cube Loans through an **MDX** Query

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read(r'..\..\config.ini')

In the MDX query we are limiting the elements from the Loan dimension to the first 20k.

While in the cube we have facts on > 1M loans, for this demo we will be using a reduced set of loans in order to assure that the succeeding steps (e.g. visualizations) will be fast.

We will not query Loans that are still running. We want only loans that have a final state. Either `defaulted`, `charged off` or `fully paid`.

We use the `execute_mdx_dataframe` method to build a dataframe from the result of the MDX query

Then we store the results in the `loans_raw` variable

In [ ]:
from TM1py import TM1Service

mdx = """
SELECT 
    NON EMPTY  
    { HEAD ( {Tm1FilterByLevel ( {Tm1SubsetAll ([Loan])} , 0 ) } , 20000 ) }  * 
    {Tm1FilterByLevel ( {Tm1SubsetAll ([LC Rating])} , 0 ) } * 
    {Tm1FilterByLevel ( {Tm1SubsetAll ([FICO Score])} , 0 ) } *
    {Tm1FilterByLevel ( {Tm1SubsetAll ([Purpose])} , 0 ) } * 
    {Tm1FilterByLevel ( {Tm1SubsetAll ([State])} , 0 ) } * 
    {Tm1FilterByLevel ( {Tm1SubsetAll ([Income To Loan Ratio])} , 0 ) } * 
    {Tm1FilterByLevel ( {Tm1SubsetAll ([Home Ownership])} , 0 ) } *
    {Tm1FilterByLevel ( {Tm1SubsetAll ([Delinquency Events])} , 0 ) } *
    {Tm1FilterByLevel ( {Tm1SubsetAll ([Time])} , 0 ) } *
    {Tm1FilterByLevel ( {Tm1SubsetAll ([Income])} , 0 ) } *
    {Tm1FilterByLevel ( {Tm1SubsetAll ([Application Type])} , 0 ) } *
    {[Loan Status].[Fully Paid], [Loan Status].[Charged Off], [Loan Status].[default]}  ON ROWS,
    {[Loans Measure].[loan_amnt], [Loans Measure].[defaulted], [Loans Measure].[int_rate],
    [Loans Measure].[num_personal_inquiries], [Loans Measure].[inquiries_in_last_12m],
    [Loans Measure].[mths_since_last_delinq], [Loans Measure].[mths_since_recent_bc_dlq], 
    [Loans Measure].[mths_since_recent_inq]} ON COLUMNS
FROM [Loans]
WHERE ([Employment].[Total Employment], [Term].[Total Term])
"""

with TM1Service(**config['tm1srv01']) as tm1:
    loans_raw = tm1.cubes.cells.execute_mdx_dataframe(mdx)

First we assess the structure of the returned data.

We print out number of rows and columns by calling the `shape` property on the loans_raw DataFrame

In [ ]:
loans_raw.shape

Now let's take a closer look at the actual data.

We print out the first 10 entries from the DataFrame with the `head` method

In [ ]:
loans_raw.head(10)

The data is flat :-(

The is an unpleasent sideeffect of the `execute_mdx_dataframe` method that we used to retrieve the data from the cube.
Before we can plot it and analyse it further, we need to bring into a shape that is more convenient.

_Sidenote:
The execute_mdx_dataframe method is optimized for performance. It loses the original shape of the cube view when it retrieves data from TM1. To retrieve data in a pivot shape use the execute_mdx_dataframe_pivot method_

## 1.2 Preprocessing

We need to rearrange the `loans_raw` DataFrame into something that is more convenient for consumption, down the road

- Arrange Measures as seperate columns

- Remove Value Column

- Set new index in DataFrame based on Loan-Id

The result of this operation we call `loans`

In [ ]:
loans = loans_raw.copy()

# Arrange measures as columns
for measure in ("defaulted", "loan_amnt", "num_personal_inquiries", "int_rate",
                "inquiries_in_last_12m", "mths_since_last_delinq", "mths_since_recent_bc_dlq",
                "mths_since_recent_inq"):
    loans[measure] = loans.apply(lambda row: row["Value"] if row["Loans Measure"] == measure else None, axis=1)

loans.drop(columns=["Value"], inplace=True)
loans.drop(columns=["Loans Measure"], inplace=True)

columns_to_remain = ["LC Rating", "FICO Score", "Purpose", "State", "Time", "Income", "Income To Loan Ratio",
                 "Home Ownership", "Delinquency Events", "Application Type"]

loans = loans.groupby(["Loan"] + columns_to_remain).sum()

for column in columns_to_remain:
    loans.reset_index(level=column, inplace=True)

Now lets print out the `shape` of the resulting dataframe

Number of entries should now be <= 20k (20k `HEAD` limit in the MDX query - Zero Suppression)

In [ ]:
loans.shape

We print out the first 5 entries with the `head` method and see how it looks.

In [ ]:
loans.head()

Now that the DataFrame has a more convenient shape. We can use built-in pandas features to answer questions about our data 

1. What percentage of loans defaulted?
2. What is the median loan size ?
3. Correlation between FICO Score and Income

In [ ]:
loans['defaulted'].value_counts(normalize=True)

In [ ]:
loans["loan_amnt"].median()

In [ ]:
loans["Income"].corr(loans["FICO Score"])

Negative correlation between income and credit-rating-score is unexpected. 

Perhaps because income is self provided by the application, and thus unreliable ?

# Part 2: Exploratory Data Analysis and Feature Selection

-----

Now we want to use **pandas** and **plotly** to 

- Get a visual overview of the dataset we are dealing with 
- Select relevant features and remove irrelavant features from the dataframe

The results of this section we will use later to build a machine learning model



## 2.1 LC Rating

_Assigned loan grade by Lending Club_

In [ ]:
bar = go.Bar(
    x=sorted(loans["LC Rating"].unique()),
    y=loans.groupby(by="LC Rating").mean()["defaulted"].values)

layout = go.Layout(
    barmode='stack',
    title="Default Rate by Rating")

data = [bar]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Almost a linear relationship between the __LC Rating__ and the __Default Rate__. < 5 % for the best ratings until up to 70 % for the worst ratings.

Towards the  lower end of the rating classes we see a higher variance between the bars. This is simply because there are less observations available in our dataset for G rated loans.

## 2.2 Purpose

_A category provided by the borrower for the loan request_

In [ ]:
bar = go.Bar(
    x=sorted(loans["Purpose"].unique()),
    y=loans.groupby(by="Purpose").mean()["defaulted"].values)

layout = go.Layout(
    barmode='stack',
    title="Default Rate by Purpose")

data = [bar]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Meaningful differences when we compare the default rates by the puporse for the Loan.

We will feed this column to the ML model further down the road.

## 2.3 State

_The state provided by the borrower in the loan application_

In [ ]:
loans_by_state = loans.groupby(by="State").mean()["defaulted"] * 100
loans_by_state.sort_values()

Significant differencs by state! Let's visualize this in a map go a better overview.

In [ ]:
data = [go.Choropleth(
    autocolorscale=True,
    locations=loans_by_state.index,
    z=loans_by_state.values,
    locationmode='USA-states',
    marker=go.choropleth.Marker(
        line=go.choropleth.marker.Line(
            color='rgb(255,255,255)',
            width=2
        )),
    colorbar=go.choropleth.ColorBar(
        title="Defaults in %")
)]

layout = go.Layout(
    title=go.layout.Title(
        text='Percentage of Loans that default'
    ),
    geo=go.layout.Geo(
        scope='usa',
        projection=go.layout.geo.Projection(type='albers usa'),
        showlakes=True,
        lakecolor='rgb(255, 255, 255)'),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Significant differences by state. 

Nebraska and Oklahoma > 30 % while Oregon has a rate of < 15 %

## 2.4 Income To Loan Ratio

_A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income._

Now let's first check if there a differences in the **Default Rate** on either side of the range

Let's calculate the default rate for **Income To Loan Ratio** below 10 and greater than 20

In [ ]:
df_temp = loans.loc[loans['Income To Loan Ratio'] < 10]

sum(df_temp["defaulted"]) / len(df_temp)

12.7 % default rate for borrowers with an **Income to Loan Ratio** below 10

In [ ]:
df_temp = loans.loc[loans['Income To Loan Ratio'] > 20]

sum(df_temp["defaulted"]) / len(df_temp)

23 % default rate for borrowers with an **Income to Loan Ratio** > 20

Judging from these two insights it seems the **Income To Loan Ratio** is a driver for the default rate.

Now, to get the full picture we group the DataFrame by the **defaulted** column and then use the `descibe` method on the **Income To Loan Ratio** column

In [ ]:
loans.groupby('defaulted')['Income To Loan Ratio'].describe()

So the mean **Income to Loan Ratio** for Fully paid loans was approx 18, while the mean **Income to Loan Ratio** for defaulted loans was approx 21. 

## 2.5 Income

_The self-reported annual income provided by the borrower during registration._

In [ ]:
bar = go.Bar(
    x=sorted(loans["Income"].unique()),
    y=loans.groupby(by="Income").mean()["defaulted"].values)

layout = go.Layout(
    barmode='stack',
    title="Default Rate by Income")

data = [bar]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

**Income** is provided by the borrower. It seems unreliable for three reasons

1. Negative correlation between self provided income and official credit rating score
2. very low default rate for income of 10k is implausible
3. Spike in default rate for income of 120k seems implausible 


Let's remove it from the dataframe and not use down the road ! We remove the column with the `drop` method

In [ ]:
loans.drop("Income", axis=1, inplace=True)

## 2.6 Application Type

_Indicates whether the loan is an individual application or a joint application with two co-borrowers_

In [ ]:
bar = go.Bar(
    x=sorted(loans["Application Type"].unique()),
    y=loans.groupby(by="Application Type").mean()["defaulted"].values)

layout = go.Layout(
    barmode='stack',
    title="Default Rate by Income")

data = [bar]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Default rate of the application types are quite different! 

In [ ]:
loans['Application Type'].value_counts(normalize=True)

However only 0.6 % of the applications are joint. So it's not going to be of much help to help to predicts potential defaults among new loans.

## 2.7 Time

_The month the loan was funded_

We use the `describe` method on the column to get an overview

In [ ]:
loans['Time'].describe()

Due to the MDX query we used, all loans are from Dec-2015. This Information is not relevant. Let's remove the column with the `drop` method

In [ ]:
loans.drop("Time", axis=1, inplace=True)

## 2.8 num_personal_inquiries

_Number of personal finance inquiries_

In [ ]:
bar = go.Bar(
    x=sorted(loans["num_personal_inquiries"].unique()),
    y=loans.groupby(by="num_personal_inquiries").mean()["defaulted"].values)

layout = go.Layout(
    barmode='stack',
    title="Default Rate by Number of Personal Finance Inquiries")

data = [bar]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Approximately linear relationship between the default rate and the Number of Personal Finance Inquiries

**Take away**: Applications with more Personal Finance Inquiries in the past have a higher default rate. 

100 % default rate for applications with 17 Finance Inquiries (probably just one or two records in the 20'000 loans we are looking at)



# Part 3: More Data Processing

-----

### Translate String columns to Numeric or Binary

Unfortunately Classifier Implementations for Python can only consume numeric **features**, so we need to translate string columns (e.g. State, Rating, purpose) into numeric and binary columns.

**FROM**

| Loan  | State  |
| :-: | :-: |
| Loan 5 | CA |
| Loan 8 | WA |


**TO**

| Loan  | State_CA | State_WA |
| :-: | :-: | :-: |
| Loan 5 | 1 | 0 |
| Loan 8 | 0 | 1 |


The pandas library has a built-in functionality to translate categorical features into numeric features. It's called `get_dummies`

The new DataFrame we will call `loans_numeric`

In [ ]:
import pandas as pd

loans_numeric = pd.get_dummies(
    loans, 
    columns=['LC Rating', 'Home Ownership', 'Purpose', 'State', 'Application Type'],
    drop_first=True)

In [ ]:
loans_numeric.shape

In the resulting DataFrame we have the same amount of rows as before the transformation, but a lot more columns. This is the result we expected.

By default Jupyter will not print all > 100 columns . The `...` represent the missing columns  

In [ ]:
loans_numeric.head()

Now we split the `loans_numeric` DataFrame into **Features** (e.g. Income, Rating) and **Class** (Defaulted or Fully Paid). 

This is necessary, so that we can use the ML frameworks that python provides

Naming of `X` and `y` is a convention

In [ ]:
X = loans_numeric.loc[:, loans_numeric.columns != "defaulted"]
y = loans_numeric["defaulted"]

### Calculate correlation between columns and default

Now that all columns are numeric values we can calculate correlations between them

_Sidenote:_
- Values between 0 and 0.3 (0 and -0.3) indicate a weak positive (negative) linear relationship
- Values between 0.3 and 0.7 (-0.3 and -0.7) indicate a moderate positive (negative) linear relationship
- Values between 0.7 and 1.0 (-0.7 and -1.0) indicate a strong positive (negative) linear relationship

In [ ]:
linear_dep = pd.DataFrame()
for col in X.columns:
    linear_dep.loc[col, 'corr'] = X[col].corr(y)
    
linear_dep['abs_corr'] = abs(linear_dep['corr'])
linear_dep.sort_values('abs_corr', ascending=False, inplace=True)

linear_dep.head(10)

# Part 4: Fit And Evaluate Maschine Learning Model

-----


## 4.1 Scale data and split into test and training sets

Before applying Machine Learning, we need to scale our data such that each feature has the same variance.

This is necessary to avoid differences in weights for classifiers like KNN.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X)
X_scaled=scaler.transform(X)

The `shape` of the DataFrame should not have been impacted by this operation! We are expecting the same amount of rows and columns in the result

In [ ]:
X_scaled.shape

Now let's print out the first entry from the DataFrame and see if the values are roughly in the same scale

In [ ]:
X_scaled[0]

### Split data set into train and test sets

Now, before we `fit` and `apply` a ML model, let's split our original dataframe into a **test** and **training** data set
This enables us to do to unbiased testing of the ML model later.

To split the data randomly we are using the `train_test_split` method from the `sklearn` library.


| Variable  | Content  |
| :-: | :-: |
| X_train | 80 % of the features |
| X_test | 20 % of the features |
| y_train | The results (Defaulted or Fully Paid) to the 80 %  |
| y_test | the results (Defaulted or Fully Paid) to the 20 %  |


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=4)

## 4.2 KNN (K Nearest Neighbors)

`fit` a KNN model. 

We provide 1 arugment: `n_neighters` (number of neighbors).
How to find the "right" value for this argument ? Try out different combinations and optimize for the desired outcome...

_Sidenote: Fitting the ML model is just 2 lines of code!_

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)

Now we apply the fitted KNN model on the test data set (`X_test`) with the `predict` method

In [ ]:
y_pred = knn_classifier.predict(X_test)

### Evaluate KNN results

We use 3 Metrics to assess the quality of the model

1. Confusion Matrix
2. Classification Report
3. ROC Curve

These metrics are all implemented in `sklearn` so we don't need to write any custom code for it

## Confusion Matrix

Calculate `confusion_matrix` from results on the test set and pretty-print visualization of confusion matrix as a `heatmap`


| Corner  | Description  |
| :-: | :-: |
| **Top Left**  | True Positives: Loan default and we predicted default |
| **Top Right** | False Negatives: Loan default and we predicted fine |
| **Bottom Left** | False Positive: Loan fine and we predicted default |
| **Bottom Right** | True Negatives: Loan fine and we predicted fine |



Perfect precission would output a matrix with all values on the diagonal...

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[1, 0])

ax = plt.subplot()
sns.heatmap(
    matrix, 
    annot=True, 
    fmt='d',
    linewidths=.5,
    cmap="YlGnBu");

# labels and titles
ax.set_xlabel('Predicted Values')
ax.set_ylabel('Actual Values')
ax.xaxis.set_ticklabels(['Defaulted', 'Fully Paid'])
ax.yaxis.set_ticklabels(['Defaulted', 'Fully Paid'])

## Classification Report

Calculate the  `classification_report`. This metric is based on the results from the confusion matrix.


| Variable  | Description  |
| :-: | :-: |
| **precision**  | What percent of the predicted defaults were correct ? |
| **recall** | What percent of the defaults did we catch ? |
| **f1 score** | Weighted average over precission and recall |
| **support** | number of records |



In [ ]:
from sklearn.metrics import classification_report

knn_report = classification_report(y_test, y_pred, output_dict=True)

print(classification_report(y_test, y_pred))

### ROC Curve

The dotted blue line is chance (flipping a coin). The orange line is our fitted classifier.

The curve is plotted by executing the model with different parameters and plotting the **FPR** against the **TPR**.
The closer the curve follows the left-hand border and then the top border of the ROC space, the more accurate the model is.

The area under the orange curve is called **AUC Score**. 
It is a generally accepted measure and convenient when comparing different models with one another.

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# predict probabilities
probs = knn_classifier.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, probs)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
# show the plot
plt.show()

So - What’s a good AUC score ?
There is no magic threshold. Higher is obviously better, but it is entirely application dependent.

## 4.3 Random Forest classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(
    n_estimators=100,
    max_features=20,
    max_depth=100,
    random_state=4)

rf_classifier.fit(X_train, y_train)

In [ ]:
y_pred = rf_classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix 
import seaborn as sns

matrix = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[1, 0])

ax = plt.subplot()
sns.heatmap(
    matrix, 
    annot=True, 
    fmt='d',
    linewidths=.5,
    cmap="YlGnBu");

# labels and titles
ax.set_xlabel('Predicted Values')
ax.set_ylabel('Actual Values')
ax.xaxis.set_ticklabels(['Defaulted', 'Fully Paid'])
ax.yaxis.set_ticklabels(['Defaulted', 'Fully Paid'])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

#rf_report = classification_report(y_test, y_pred, output_dict=True)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# predict probabilities
probs = rf_classifier.predict_proba(X_test)

# keep probabilities for the positive outcome only
probs = probs[:, 1]

# calculate AUC
auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, probs)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(fpr, tpr, marker='.')
plt.show()

Overall, the Random Forests provides better results according to the AUC measure.

### Plot features importance

the random forest can provide quantitative measures how relevant the features in our dataframe where.
This makese the results more transparent.

In [ ]:
import pandas as pd

feature_importances = pd.DataFrame(
    rf_classifier.feature_importances_,
    index = X.columns,
    columns=['importance']).sort_values('importance', ascending=False)

feature_importances.head(10)

# Other

Things that didn't make it into the demo.

-----

## 2.6 Home Ownership

The home ownership status provided by the borrower during registration. 

Possible values are: RENT, OWN, MORTGAGE, OTHER.

In [ ]:
bar = go.Bar(
    x=sorted(loans["Home Ownership"].unique()),
    y=loans.groupby(by="Home Ownership").mean()["defaulted"].values)

layout = go.Layout(
    barmode='stack',
    title="Default Rate by Home Ownership Type")

data = [bar]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## 2.2 FICO Score

A credit score created by the Fair Isaac Corporation

In [ ]:
defaulted = loans.loc[loans['defaulted'] == 1]['FICO Score']
fine = loans.loc[loans['defaulted'] == 0]['FICO Score']

hist1 = go.Histogram(
    x=fine,
    name="Fully Paid"
)
hist2 = go.Histogram(
    x=defaulted,
    name="Defaulted"
)

data = [hist1, hist2]
layout = go.Layout(
    barmode="stack",
    title="Histograms on FICO Score")
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

## Logistic Regression

In [ ]:
from sklearn.linear_model import SGDClassifier

logrec_classifier = SGDClassifier(
    loss='log', 
    max_iter=1000, 
    tol=1e-3, 
    random_state=1, 
    warm_start=True,
    alpha=0.01, 
    penalty='l2')

logrec_classifier.fit(X_train, y_train)

In [ ]:
y_pred = logrec_classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix 

matrix = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[1, 0])
print(matrix)

### Print classification report

**precision** - What percent of the predicted defaults that were correct ?

**recall** – What percent of the defaults did we catch ?

**f1 score** – Weighted average over precission and recall

**support** - number of records

In [ ]:
from sklearn.metrics import classification_report

logrec_report = classification_report(y_test, y_pred, output_dict=True)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# predict probabilities
probs = logrec_classifier.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, probs)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
# show the plot
plt.show()

## PCA Dimension Reduction and Visualization in 2D

_PCA is essentially a method that reduces the dimension of the feature space in such a way that new variables are orthogonal to each other (i.e. they are independent or not correlated)._


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(X_train)
principal_df = pd.DataFrame(
    data = principal_components, 
    columns = ['principal component 1', 'principal component 2'])

In [ ]:
data = [go.Scatter(
    x = principal_df["principal component 1"],
    y = principal_df["principal component 2"],
    mode = 'markers',
    marker = dict(
        color = y_train.values
        ),
    )]

py.iplot(data)